In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from glob import glob
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=15, threads_per_worker=1)
client  = Client(cluster, asynchronous=True)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 15
Total threads: 15,Total memory: 125.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38383,Workers: 15
Dashboard: http://127.0.0.1:8787/status,Total threads: 15
Started: Just now,Total memory: 125.57 GiB
Comm: tcp://127.0.0.1:43155,Total threads: 1
Dashboard: http://127.0.0.1:35145/status,Memory: 8.37 GiB
Nanny: tcp://127.0.0.1:46793,


In [2]:
masks = xr.open_dataset('data/REFORECASTS_MASKS.nc')
lon,lat = np.meshgrid(masks.lon,masks.lat)
tropicalcoords   = pd.DataFrame((lon[0,:],np.zeros(len(lon[0,:]))), index=['lon','lat']).T
lonn = []
lons = []
for i in range(len(masks.lat)):
    try:
        x = masks.coastmask_north.where(masks.coastmask_north==1).isel(lat=i).dropna('lon')[-1].lon.item()
        lonn.append(x)
    except:
        lonn.append(np.nan)
        pass
    
    try:
        x = masks.coastmask_south.where(masks.coastmask_south==1).isel(lat=i).dropna('lon')[-1].lon.item()
        lons.append(x)
    except:
        lons.append(np.nan)
        pass
coastnorthcoords = pd.DataFrame((np.array(lonn),lat[:,0]),index=['lon','lat']).T.dropna()
coastsouthcoords = pd.DataFrame((np.array(lons),lat[:,0]),index=['lon','lat']).T.dropna()

coastnorthcoords.index = coastnorthcoords.lat
coastsouthcoords.index = coastsouthcoords.lat

In [50]:
dirpaths = sorted(glob('data/S2S/reforecasts/*'))[:-40]
hovmoller_tropical   = []
hovmoller_coastnorth = []
hovmoller_coastsouth = []
for directory in dirpaths:
    print(directory)
    paths = glob(directory+'/*_SSH.nc')
    data1  = []
    for p in paths:
        d = xr.open_dataset(p).squeeze()
        d = d.sortby('lat').sortby('lon').sel({'lon':slice(220,290),'lat':slice(-40,40)})
        d = d.rename({'time':'leadtime'})
        d.coords['inittime'] = d.leadtime[0].values
        d['leadtime'] = np.arange(len(d.leadtime))
        data1.append(d['dslm'])
    data1     = xr.concat(data1, 'inittime').drop('depth').sortby('inittime').sortby('leadtime')
    
    paths = glob(directory+'/*_300T.nc')
    data2  = []
    for p in paths:
        d = xr.open_dataset(p).squeeze()
        d = d.sortby('lat').sortby('lon').sel({'lon':slice(220,290),'lat':slice(-40,40)})
        d = d.rename({'time':'leadtime'})
        d.coords['inittime'] = d.leadtime[0].values
        d['leadtime'] = np.arange(len(d.leadtime))
        data2.append(d['param18.4.10'])
    data2     = xr.concat(data2, 'inittime').drop('depth').sortby('inittime').sortby('leadtime')
    data = xr.merge([data1,data2.ffill('lon')])
    
    tropical = data.where(masks.tropicalmask).mean('lat').dropna('lon').sortby('lon')
    tropical.coords['index'] = ('lon',np.arange(len(tropical.lon)))
    # tropical = tropical.to_dataset(name=var[1]+'_tropical')
    
    coast_north = data.where(masks.coastmask_north).mean('lon').dropna('lat').sortby('lat')
    coast_north.coords['index'] = ('lat', 1+tropical.index[-1].item()+np.arange(len(coast_north.lat)))
    # coast_north = coast_north.to_dataset(name=var[1]+'_coastnorth')
    
    coast_south = data.where(masks.coastmask_south).mean('lon').dropna('lat').sortby('lat', ascending=False)
    coast_south.coords['index'] = ('lat', 1+tropical.index[-1].item()+np.arange(len(coast_south.lat)))
    # coast_south = coast_south.to_dataset(name=var[1]+'_coastsouth')
    
    hovmoller_tropical.append(tropical.rename({'dslm':'zos', 'param18.4.10':'T300'}))
    hovmoller_coastnorth.append(coast_north.rename({'dslm':'zos', 'param18.4.10':'T300'}))
    hovmoller_coastsouth.append(coast_south.rename({'dslm':'zos', 'param18.4.10':'T300'}))
    
print('joining')
hovmoller_tropical   = xr.concat(hovmoller_tropical, 'inittime').sortby('inittime')
hovmoller_coastnorth = xr.concat(hovmoller_coastnorth, 'inittime').sortby('inittime')
hovmoller_coastsouth = xr.concat(hovmoller_coastsouth, 'inittime').sortby('inittime')

data/S2S/reforecasts/2020-01-06
data/S2S/reforecasts/2020-01-09
data/S2S/reforecasts/2020-01-13
data/S2S/reforecasts/2020-01-16
data/S2S/reforecasts/2020-01-20
data/S2S/reforecasts/2020-01-23
data/S2S/reforecasts/2020-01-27
data/S2S/reforecasts/2020-01-30
data/S2S/reforecasts/2020-02-03
data/S2S/reforecasts/2020-02-06
data/S2S/reforecasts/2020-02-10
data/S2S/reforecasts/2020-02-13
data/S2S/reforecasts/2020-02-17
data/S2S/reforecasts/2020-02-20
data/S2S/reforecasts/2020-02-24
data/S2S/reforecasts/2020-02-27
data/S2S/reforecasts/2020-03-02
data/S2S/reforecasts/2020-03-05
data/S2S/reforecasts/2020-03-09
data/S2S/reforecasts/2020-03-12
data/S2S/reforecasts/2020-03-16
data/S2S/reforecasts/2020-03-19
data/S2S/reforecasts/2020-03-23
data/S2S/reforecasts/2020-03-26
data/S2S/reforecasts/2020-03-30
data/S2S/reforecasts/2020-04-02
data/S2S/reforecasts/2020-04-06
data/S2S/reforecasts/2020-04-09
data/S2S/reforecasts/2020-04-13
data/S2S/reforecasts/2020-04-16
data/S2S/reforecasts/2020-04-20
data/S2S

In [ ]:
hovmoller_tropical   = hovmoller_tropical.swap_dims({'lon':'index'})
hovmoller_coastnorth = hovmoller_coastnorth.swap_dims({'lat':'index'})
hovmoller_coastsouth = hovmoller_coastsouth.swap_dims({'lat':'index'})

In [52]:
hovmoller_tropical.coords['lat']    = ('index', np.zeros(len(tropical.lon)))
hovmoller_coastnorth.coords['lon']  = ('index', coastnorthcoords.loc[hovmoller_coastnorth.lat.values].lon.values)
hovmoller_coastsouth.coords['lon']  = ('index', coastsouthcoords.loc[hovmoller_coastsouth.lat.values].lon.values)

In [58]:

def haversine(p1,p2):
    """
    Given two points with lat,lon coordinates, compute the distance
    between those points on the surface of the sphere with the haversine formula
    Args:
        p1 (tuple): first point lat,lon
        p2 (tuple): last point lat,lon

    Returns:
        float: distance
    """
    lat1,lon1 = p1
    lat2,lon2 = p2

    lon1,lon2,lat1,lat2 = map(np.deg2rad, [lon1,lon2,lat1,lat2])

    dlon = lon2-lon1
    dlat = lat2-lat1

    a = np.sin(dlat/2)**2+np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2*np.arcsin(np.sqrt(a))
    r = 6371
    return c*r

def distances(data):
    distance = []
    for i in range(len(data.lon.values)-1):
        lon1,lat1 = data.lon.values[i], data.lat.values[i]
        lon2,lat2 = data.lon.values[i+1], data.lat.values[i+1]
        dist = haversine(p1=(lat2,lon2), p2=(lat1,lon1))
        distance.append(dist)
    distance = np.hstack([np.array([0]),np.array(distance)])
    distance = np.cumsum(distance)
    return distance
    
    

In [59]:
tropicaldistance   = distances(hovmoller_tropical)
coastnorthdistance = distances(hovmoller_coastnorth) + tropicaldistance[-1]
coastsouthdistance = distances(hovmoller_coastsouth) + tropicaldistance[-1]
hovmoller_tropical.coords['distance']    = ('index', tropicaldistance)
hovmoller_coastnorth.coords['distance']  = ('index', tropicaldistance[-1]+coastnorthdistance)
hovmoller_coastsouth.coords['distance']  = ('index', tropicaldistance[-1]+coastsouthdistance)

In [72]:
hovmoller_tropical.to_netcdf('data/S2S/HOVMOLLERS/reforecasts_tropical.nc')
hovmoller_coastnorth.to_netcdf('data/S2S/HOVMOLLERS/reforecasts_coastnorth.nc')
hovmoller_coastsouth.to_netcdf('data/S2S/HOVMOLLERS/reforecasts_coastsouth.nc')

In [73]:
client.shutdown()